In [49]:
# Step 1: Import All Required Libraries and Data Loading
import pandas as pd
import numpy as np
import os
import kagglehub
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE, SMOTENC
from autogluon.tabular import TabularPredictor, TabularDataset
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")

# Load datasets
train = pd.read_csv('train.csv', index_col='id')
test = pd.read_csv('test.csv', index_col='id')

path_rakesh = kagglehub.dataset_download("rakeshkapilavai/extrovert-vs-introvert-behavior-data")
df_rakesh = pd.read_csv(os.path.join(path_rakesh, 'personality_dataset.csv'))
train_combined = pd.concat([train, df_rakesh], ignore_index=True)

# Remove duplicates
train_clean = train_combined.drop_duplicates(keep='first').copy()
print(f"Cleaned train shape: {train_clean.shape}")
print(f"Test shape: {test.shape}")
print(f"Class distribution:\n{train_clean['Personality'].value_counts()}")


All libraries imported successfully!
Cleaned train shape: (21036, 8)
Test shape: (6175, 7)
Class distribution:
Personality
Extrovert    15116
Introvert     5920
Name: count, dtype: int64


In [50]:
# Step 2: k-NN Imputation for Missing Values
numeric_cols = ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside',
                'Friends_circle_size', 'Post_frequency']
cat_cols = ['Stage_fear', 'Drained_after_socializing']

print("Missing values before imputation:")
print("Train:", train_clean[numeric_cols + cat_cols].isnull().sum())
print("Test:", test[numeric_cols + cat_cols].isnull().sum())

# k-NN imputation for numerics
imputer = KNNImputer(n_neighbors=5)
train_clean.loc[:, numeric_cols] = imputer.fit_transform(train_clean[numeric_cols])
test.loc[:, numeric_cols] = imputer.transform(test[numeric_cols])

# Simple mode fill for categoricals
for col in cat_cols:
    if col in train_clean.columns:
        mode_val = train_clean[col].mode()[0] if not train_clean[col].mode().empty else 'No'
        train_clean.loc[:, col] = train_clean[col].fillna(mode_val)
        test.loc[:, col] = test[col].fillna(mode_val)

print("Missing values after imputation:")
print("Train:", train_clean.isnull().sum().sum())
print("Test:", test.isnull().sum().sum())


Missing values before imputation:
Train: Time_spent_Alone             1251
Social_event_attendance      1241
Going_outside                1531
Friends_circle_size          1129
Post_frequency               1327
Stage_fear                   1966
Drained_after_socializing    1200
dtype: int64
Test: Time_spent_Alone             425
Social_event_attendance      397
Going_outside                466
Friends_circle_size          350
Post_frequency               408
Stage_fear                   598
Drained_after_socializing    432
dtype: int64
Missing values after imputation:
Train: 0
Test: 0


In [51]:
# Step 3: Label Encoding for Binary Categoricals
# Check unique values first
print("Unique values in categorical columns:")
for col in cat_cols:
    if col in train_clean.columns:
        print(f"{col}: {train_clean[col].unique()}")

# Create mapping based on actual values
for col in cat_cols:
    if col in train_clean.columns:
        unique_vals = list(train_clean[col].unique())
        if 'Yes' in unique_vals and 'No' in unique_vals:
            binary_map = {'Yes': 1, 'No': 0}
        else:
            # Create numeric mapping for whatever values exist
            binary_map = {val: i for i, val in enumerate(unique_vals)}

        train_clean.loc[:, col] = train_clean[col].map(binary_map).fillna(0)
        test.loc[:, col] = test[col].map(binary_map).fillna(0)

print("Encoded categorical sample:")
print(train_clean[cat_cols].head())


Unique values in categorical columns:
Stage_fear: ['No' 'Yes']
Drained_after_socializing: ['No' 'Yes']
Encoded categorical sample:
  Stage_fear Drained_after_socializing
0          0                         0
1          0                         0
2          1                         0
3          0                         0
4          0                         0


In [52]:
# Step 4: Standard Scaling with Log Transformation
# Apply log transform to skewed features (positive values)
skewed_cols = ['Time_spent_Alone', 'Post_frequency']

# Check data ranges first
print("Data ranges before scaling:")
for col in numeric_cols:
    print(f"{col}: {train_clean[col].min():.2f} to {train_clean[col].max():.2f}")

# Apply log transform (ensuring positive values)
for col in skewed_cols:
    if col in train_clean.columns:
        train_clean.loc[:, col] = np.log1p(np.abs(train_clean[col]))
        test.loc[:, col] = np.log1p(np.abs(test[col]))

# Standard scaling
scaler = StandardScaler()
train_clean.loc[:, numeric_cols] = scaler.fit_transform(train_clean[numeric_cols])
test.loc[:, numeric_cols] = scaler.transform(test[numeric_cols])

print("Data ranges after scaling:")
for col in numeric_cols:
    print(f"{col}: {train_clean[col].min():.2f} to {train_clean[col].max():.2f}")


Data ranges before scaling:
Time_spent_Alone: 0.00 to 11.00
Social_event_attendance: 0.00 to 10.00
Going_outside: 0.00 to 7.00
Friends_circle_size: 0.00 to 15.00
Post_frequency: 0.00 to 10.00
Data ranges after scaling:
Time_spent_Alone: -1.61 to 1.76
Social_event_attendance: -1.84 to 1.76
Going_outside: -1.87 to 1.48
Friends_circle_size: -1.84 to 1.70
Post_frequency: -2.37 to 1.23


In [53]:
# Step 5: Skip SMOTE - Use Original Imbalanced Data with Class Weights
X_train = train_clean.drop('Personality', axis=1)
y_train = train_clean['Personality']

print("Original class distribution (keeping imbalanced):")
print(y_train.value_counts())

# Calculate class weight ratio for later use in models
extrovert_count = len(y_train[y_train == 'Extrovert'])
introvert_count = len(y_train[y_train == 'Introvert'])
class_weight_ratio = extrovert_count / introvert_count

print(f"Class weight ratio (Extrovert/Introvert): {class_weight_ratio:.2f}")

# Use original data without SMOTE
train_balanced = train_clean.copy()  # Keep original imbalanced data
print(f"Dataset shape (no SMOTE): {train_balanced.shape}")

# Validate data integrity
print("\nData validation:")
print(f"Any NaN in data: {train_balanced.isnull().any().any()}")
print(f"Any infinite values: {np.isinf(train_balanced.select_dtypes(include=[np.number])).any().any()}")


Original class distribution (keeping imbalanced):
Personality
Extrovert    15116
Introvert     5920
Name: count, dtype: int64
Class weight ratio (Extrovert/Introvert): 2.55
Dataset shape (no SMOTE): (21036, 8)

Data validation:
Any NaN in data: False
Any infinite values: False


In [54]:
# Step 6: Advanced Outlier Handling with Isolation Forest
iso = IsolationForest(contamination=0.01, random_state=42)
outlier_mask = iso.fit_predict(train_balanced.drop('Personality', axis=1)) == 1
train_balanced_clean = train_balanced[outlier_mask].copy()

print(f"Shape after outlier removal: {train_balanced_clean.shape}")
print(f"Outliers removed: {len(train_balanced) - len(train_balanced_clean)}")

# Check class distribution after outlier removal
print("Class distribution after outlier removal:")
print(train_balanced_clean['Personality'].value_counts())


Shape after outlier removal: (20825, 8)
Outliers removed: 211
Class distribution after outlier removal:
Personality
Extrovert    15105
Introvert     5720
Name: count, dtype: int64


In [58]:
# Step 7: Enhanced Feature Engineering and Selection
# Add basic interaction features
if 'Stage_fear' in train_balanced_clean.columns and 'Time_spent_Alone' in train_balanced_clean.columns:
    train_balanced_clean['Fear_Alone_Inter'] = train_balanced_clean['Stage_fear'] * train_balanced_clean['Time_spent_Alone']
    test['Fear_Alone_Inter'] = test['Stage_fear'] * test['Time_spent_Alone']

# Add social activity score
social_cols = ['Social_event_attendance', 'Going_outside', 'Post_frequency']
available_social_cols = [col for col in social_cols if col in train_balanced_clean.columns]
if available_social_cols:
    train_balanced_clean['Social_Activity_Score'] = train_balanced_clean[available_social_cols].sum(axis=1)
    test['Social_Activity_Score'] = test[available_social_cols].sum(axis=1)

# NEW: Advanced psychological feature interactions
# Social confidence vs fear ratio
if 'Social_event_attendance' in train_balanced_clean.columns and 'Stage_fear' in train_balanced_clean.columns:
    train_balanced_clean['Social_Fear_Ratio'] = train_balanced_clean['Social_event_attendance'] / (train_balanced_clean['Stage_fear'] + 1)
    test['Social_Fear_Ratio'] = test['Social_event_attendance'] / (test['Stage_fear'] + 1)

# Introversion tendency score (time alone vs social activity)
if 'Time_spent_Alone' in train_balanced_clean.columns and 'Social_event_attendance' in train_balanced_clean.columns:
    train_balanced_clean['Introversion_Score'] = train_balanced_clean['Time_spent_Alone'] - train_balanced_clean['Social_event_attendance']
    test['Introversion_Score'] = test['Time_spent_Alone'] - test['Social_event_attendance']

# Digital vs physical social preference
if 'Post_frequency' in train_balanced_clean.columns and 'Going_outside' in train_balanced_clean.columns:
    train_balanced_clean['Digital_Social_Ratio'] = train_balanced_clean['Post_frequency'] / (train_balanced_clean['Going_outside'] + 1)
    test['Digital_Social_Ratio'] = test['Post_frequency'] / (test['Going_outside'] + 1)

# Social network size vs activity alignment
if 'Friends_circle_size' in train_balanced_clean.columns and 'Social_event_attendance' in train_balanced_clean.columns:
    train_balanced_clean['Friends_Activity_Ratio'] = train_balanced_clean['Friends_circle_size'] / (train_balanced_clean['Social_event_attendance'] + 1)
    test['Friends_Activity_Ratio'] = test['Friends_circle_size'] / (test['Social_event_attendance'] + 1)

print(f"Features after enhanced engineering: {list(train_balanced_clean.columns)}")
print(f"Shape after feature engineering: {train_balanced_clean.shape}")

# Feature selection with RFE (increased to capture more features)
base_estimator = LogisticRegression(solver='liblinear', random_state=42, max_iter=1000)
n_features_to_select = min(15, len(train_balanced_clean.columns)-1)  # Increased from 10 to 15
rfe = RFE(estimator=base_estimator, n_features_to_select=n_features_to_select, step=1)
rfe.fit(train_balanced_clean.drop('Personality', axis=1), train_balanced_clean['Personality'])
selected_features = train_balanced_clean.columns.drop('Personality')[rfe.support_]
print(f"Selected features ({len(selected_features)}): {list(selected_features)}")

# Filter datasets to selected features
train_optimized = train_balanced_clean[selected_features.tolist() + ['Personality']].copy()
test_optimized = test[selected_features].copy()

print(f"Final optimized shape - Train: {train_optimized.shape}, Test: {test_optimized.shape}")


Features after enhanced engineering: ['Time_spent_Alone', 'Stage_fear', 'Social_event_attendance', 'Going_outside', 'Drained_after_socializing', 'Friends_circle_size', 'Post_frequency', 'Personality', 'Fear_Alone_Inter', 'Social_Activity_Score', 'Social_Fear_Ratio', 'Introversion_Score', 'Digital_Social_Ratio', 'Friends_Activity_Ratio']
Shape after feature engineering: (20825, 14)
Selected features (13): ['Time_spent_Alone', 'Stage_fear', 'Social_event_attendance', 'Going_outside', 'Drained_after_socializing', 'Friends_circle_size', 'Post_frequency', 'Fear_Alone_Inter', 'Social_Activity_Score', 'Social_Fear_Ratio', 'Introversion_Score', 'Digital_Social_Ratio', 'Friends_Activity_Ratio']
Final optimized shape - Train: (20825, 14), Test: (6175, 13)


In [59]:
# Step 8: Enhanced AutoGluon Training with Advanced Configuration
# Create train/validation split for evaluation
X_train_final, X_val_final, y_train_final, y_val_final = train_test_split(
    train_optimized.drop('Personality', axis=1),
    train_optimized['Personality'],
    test_size=0.2,
    random_state=42,
    stratify=train_optimized['Personality']
)

# Create TabularDataset
train_ag = TabularDataset(train_optimized)
val_ag = TabularDataset(pd.concat([X_val_final, y_val_final], axis=1))

# Enhanced configuration with more sophisticated models and hyperparameters
fit_config = {
    'time_limit': 14400,  # 4 hours for deeper search
    'presets': 'best_quality',  # Upgraded from medium_quality
    'hyperparameters': {
        'XGB': [
            {
                'n_estimators': 500,
                'max_depth': 8,
                'learning_rate': 0.05,
                'scale_pos_weight': class_weight_ratio,
                'subsample': 0.8,
                'colsample_bytree': 0.8,
                'min_child_weight': 3
            },
            {
                'n_estimators': 800,
                'max_depth': 10,
                'learning_rate': 0.03,
                'scale_pos_weight': class_weight_ratio,
                'subsample': 0.9,
                'colsample_bytree': 0.9
            }
        ],
        'RF': [
            {'n_estimators': 300, 'max_depth': 20, 'class_weight': 'balanced', 'max_features': 'sqrt'},
            {'n_estimators': 500, 'max_depth': 25, 'class_weight': 'balanced', 'max_features': 'log2'}
        ],
        'CAT': [
            {
                'iterations': 1000,
                'learning_rate': 0.03,
                'depth': 10,
                'class_weights': [1, class_weight_ratio],
                'border_count': 254
            },
            {
                'iterations': 1500,
                'learning_rate': 0.02,
                'depth': 12,
                'class_weights': [1, class_weight_ratio]
            }
        ],
        'LR': [
            {'class_weight': 'balanced', 'C': 0.1, 'solver': 'liblinear'},
            {'class_weight': 'balanced', 'C': 1, 'solver': 'lbfgs'},
            {'class_weight': 'balanced', 'C': 10, 'solver': 'liblinear'}
        ],
        'KNN': [
            {'n_neighbors': 7, 'weights': 'distance', 'metric': 'minkowski'},
            {'n_neighbors': 12, 'weights': 'distance', 'metric': 'manhattan'},
            {'n_neighbors': 20, 'weights': 'distance', 'metric': 'euclidean'}
        ],
        'NN_TORCH': [
            {'num_epochs': 300, 'learning_rate': 0.01, 'activation': 'relu', 'dropout_prob': 0.2},
            {'num_epochs': 500, 'learning_rate': 0.005, 'activation': 'tanh', 'dropout_prob': 0.3}
        ]
    },
    'num_bag_folds': 8,   # Increased bagging
    'num_bag_sets': 2,    # Multiple bag sets for diversity
    'num_stack_levels': 3, # Deeper stacking
    'ag_args_fit': {
        'num_gpus': 0,
        'verbosity': 3
    },
    'dynamic_stacking': True,
    'auto_stack': True,
    'hyperparameter_tune_kwargs': {'num_trials': 30, 'scheduler': 'local', 'searcher': 'auto'}  # Auto hyperparameter tuning
}

print("Starting Enhanced AutoGluon training...")
print(f"Training on {len(train_optimized)} samples with {len(selected_features)} features")

predictor = TabularPredictor(label='Personality', eval_metric='accuracy', verbosity=3)

try:
    predictor.fit(train_ag, **fit_config)
    print("Enhanced training completed successfully!")

    # Get detailed leaderboard
    leaderboard = predictor.leaderboard(silent=True)
    print("Enhanced Model Leaderboard:")
    print(leaderboard.head(10))

    # Validate on holdout with detailed metrics
    val_predictions = predictor.predict(val_ag.drop('Personality', axis=1))
    val_probabilities = predictor.predict_proba(val_ag.drop('Personality', axis=1))
    val_accuracy = accuracy_score(y_val_final, val_predictions)

    print(f"Enhanced Validation Accuracy: {val_accuracy:.6f}")

    # Check class distribution in predictions
    pred_dist = pd.Series(val_predictions).value_counts()
    actual_dist = pd.Series(y_val_final).value_counts()
    print("Validation prediction vs actual distribution:")
    print(f"Predictions: {pred_dist}")
    print(f"Actual: {actual_dist}")

    # Feature importance from best model
    feature_importance = predictor.feature_importance(train_ag)
    print("\nTop 10 Most Important Features:")
    print(feature_importance.head(10))

except Exception as e:
    print(f"Enhanced training failed: {e}")
    print("Falling back to previous configuration...")

    # Fallback to previous working config
    fallback_config = {
        'time_limit': 7200,
        'presets': 'medium_quality',
        'hyperparameters': {
            'RF': {'n_estimators': 100, 'class_weight': 'balanced'},
            'XGB': {'n_estimators': 100, 'max_depth': 6, 'learning_rate': 0.1, 'scale_pos_weight': class_weight_ratio},
            'LR': {'class_weight': 'balanced'},
        },
        'num_bag_folds': 5,
        'num_bag_sets': 1,
        'num_stack_levels': 2,
        'ag_args_fit': {'num_gpus': 0, 'verbosity': 3}
    }

    predictor = TabularPredictor(label='Personality', eval_metric='accuracy', verbosity=3)
    predictor.fit(train_ag, **fallback_config)
    val_predictions = predictor.predict(val_ag.drop('Personality', axis=1))
    val_accuracy = accuracy_score(y_val_final, val_predictions)
    print(f"Fallback Validation Accuracy: {val_accuracy:.6f}")


No path specified. Models will be saved in: "AutogluonModels/ag-20250721_125522"
Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
GPU Count:          0
Memory Avail:       9.66 GB / 12.67 GB (76.2%)
Disk Space Avail:   66.56 GB / 107.72 GB (61.8%)
Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'ag_args_fit': {'num_gpus': 0, 'verbosity': 3},
 'auto_stack': True,
 'hyperparameter_tune_kwargs': {'num_trials': 30,
                                'scheduler': 'local',
                                'searcher': 'auto'},
 'num_bag_folds': 8,
 'num_bag_sets': 2,
 'num_stack_levels': 3}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag

Starting Enhanced AutoGluon training...
Training on 20825 samples with 13 features


	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 2 features to boolean dtype as they only contain 2 unique values.
			Original Features (exact raw dtype, raw dtype):
				('float64', 'float') : 9 | ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside', 'Friends_circle_size', 'Post_frequency', ...]
				('object', 'object') : 4 | ['Stage_fear', 'Drained_after_socializing', 'Fear_Alone_Inter', 'Social_Fear_Ratio']
			Types of features in original data (raw dtype, special dtypes):
				('float', [])  : 9 | ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside', 'Friends_circle_size', 'Post_frequency', ...]
				('object', []) : 4 | ['Stage_fear', 'Drained_after_socializing', 'Fear_Alone_Inter', 'Social_Fear_Ratio']
			Types of features in processed data (raw dtype, special dtypes):
				('float', [])     : 9 | ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside', 'Friends_circle_size', 'Post_frequency', ...]
				('int', ['bool']) : 2 | 

  0%|          | 0/30 [00:00<?, ?it/s]

Loading: /content/AutogluonModels/ag-20250721_125522/ds_sub_fit/sub_fit_ho/models/CatBoost_BAG_L1/dataset_train.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/ds_sub_fit/sub_fit_ho/models/CatBoost_BAG_L1/dataset_val.pkl
Saving /content/AutogluonModels/ag-20250721_125522/ds_sub_fit/sub_fit_ho/models/CatBoost_BAG_L1/T1/utils/model_template.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/ds_sub_fit/sub_fit_ho/models/CatBoost_BAG_L1/T1/utils/model_template.pkl
	Fitting 16 child models (S1F1 - S2F8) | Fitting with SequentialLocalFoldFittingStrategy
`import catboost` failed. A quick tip is to install via `pip install autogluon.tabular[catboost]==1.3.1`.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/autogluon/common/utils/try_import.py", line 68, in try_import_catboost
    import catboost
ModuleNotFoundError: No module named 'catboost'

The above exception was the direct cause of the following exception:

Traceback (most recent call last

...

Enhanced training completed successfully!
Enhanced Model Leaderboard:
                        model  score_val eval_metric  pred_time_val  \
0         WeightedEnsemble_L2   0.964562    accuracy       6.030915   
1  NeuralNetTorch_2_BAG_L1/T1   0.964370    accuracy       0.697826   
2    NeuralNetTorch_BAG_L1/T1   0.964322    accuracy       0.689823   
3  NeuralNetTorch_2_BAG_L1/T2   0.964322    accuracy       1.509497   
4    NeuralNetTorch_BAG_L1/T2   0.964226    accuracy       3.130223   
5         XGBoost_2_BAG_L1/T2   0.963938    accuracy       0.423703   
6              XGBoost_BAG_L1   0.963890    accuracy       0.332353   
7         XGBoost_2_BAG_L1/T3   0.963842    accuracy       0.479789   
8         XGBoost_2_BAG_L1/T5   0.963794    accuracy       0.405962   
9       LinearModel_BAG_L1/T2   0.963649    accuracy       0.361423   

      fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  \
0  1532.723500                0.003546           1.050594            2   


Loading: /content/AutogluonModels/ag-20250721_125522/models/NeuralNetTorch_2_BAG_L1/T2/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/NeuralNetTorch_BAG_L1/T1/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/NeuralNetTorch_BAG_L1/T2/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/WeightedEnsemble_L2/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/NeuralNetTorch_2_BAG_L1/T1/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/NeuralNetTorch_2_BAG_L1/T2/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/NeuralNetTorch_BAG_L1/T1/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/NeuralNetTorch_BAG_L1/T2/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/WeightedEnsemble_L2/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/WeightedEnsemble_L2/model.pkl
Computing feature importance via permutation shuffling for 

Enhanced Validation Accuracy: 0.965186
Validation prediction vs actual distribution:
Predictions: Personality
Extrovert    3024
Introvert    1141
Name: count, dtype: int64
Actual: Personality
Extrovert    3021
Introvert    1144
Name: count, dtype: int64


Loading: /content/AutogluonModels/ag-20250721_125522/models/NeuralNetTorch_2_BAG_L1/T2/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/NeuralNetTorch_BAG_L1/T1/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/NeuralNetTorch_BAG_L1/T2/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/WeightedEnsemble_L2/model.pkl
	665.14s	= Expected runtime (133.03s per shuffle set)
Loading: /content/AutogluonModels/ag-20250721_125522/models/NeuralNetTorch_2_BAG_L1/T1/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/NeuralNetTorch_2_BAG_L1/T2/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/NeuralNetTorch_BAG_L1/T1/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/NeuralNetTorch_BAG_L1/T2/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/WeightedEnsemble_L2/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/NeuralNetTorch_2_BAG_L1/T1/model.pk


Top 10 Most Important Features:
                           importance    stddev   p_value  n  p99_high  \
Time_spent_Alone              0.00096  0.000477  0.005429  5  0.001943   
Social_Activity_Score         0.00060  0.000469  0.022956  5  0.001566   
Fear_Alone_Inter              0.00040  0.000141  0.001599  5  0.000691   
Drained_after_socializing     0.00028  0.000303  0.053969  5  0.000905   
Post_frequency                0.00024  0.000297  0.072352  5  0.000851   
Social_Fear_Ratio             0.00024  0.000297  0.072352  5  0.000851   
Introversion_Score            0.00020  0.000374  0.149007  5  0.000970   
Going_outside                 0.00020  0.000245  0.070964  5  0.000704   
Stage_fear                    0.00012  0.000110  0.035242  5  0.000346   
Friends_Activity_Ratio        0.00008  0.000110  0.088904  5  0.000306   

                            p99_low  
Time_spent_Alone          -0.000023  
Social_Activity_Score     -0.000366  
Fear_Alone_Inter           0.000109  


In [60]:
# Step 9: Generate Final Predictions and Submission
# Generate predictions on test set using the enhanced predictor
test_ag = TabularDataset(test_optimized)
test_predictions = predictor.predict(test_ag)
test_probabilities = predictor.predict_proba(test_ag)

print("Enhanced AutoGluon predictions generated successfully!")
print("Test prediction distribution:")
print(pd.Series(test_predictions).value_counts())

# Show confidence distribution
if len(test_probabilities.columns) > 1:
    confidence_scores = test_probabilities.max(axis=1)
    print(f"Prediction confidence - Mean: {confidence_scores.mean():.4f}, Min: {confidence_scores.min():.4f}, Max: {confidence_scores.max():.4f}")

# Create submission
submission = pd.DataFrame({
    'id': test.index,
    'Personality': test_predictions
})

submission.to_csv('enhanced_submission.csv', index=False)
print("Enhanced submission file 'enhanced_submission.csv' created successfully!")
print(f"Submission shape: {submission.shape}")
print("Sample predictions:")
print(submission.head(10))

# Save probabilities for potential threshold optimization
test_probabilities.index = test.index
test_probabilities.to_csv('test_probabilities.csv')
print("Test probabilities saved to 'test_probabilities.csv'")


Loading: /content/AutogluonModels/ag-20250721_125522/models/NeuralNetTorch_2_BAG_L1/T1/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/NeuralNetTorch_2_BAG_L1/T2/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/NeuralNetTorch_BAG_L1/T1/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/NeuralNetTorch_BAG_L1/T2/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/WeightedEnsemble_L2/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/NeuralNetTorch_2_BAG_L1/T1/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/NeuralNetTorch_2_BAG_L1/T2/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/NeuralNetTorch_BAG_L1/T1/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/NeuralNetTorch_BAG_L1/T2/model.pkl
Loading: /content/AutogluonModels/ag-20250721_125522/models/WeightedEnsemble_L2/model.pkl


Enhanced AutoGluon predictions generated successfully!
Test prediction distribution:
Personality
Extrovert    4615
Introvert    1560
Name: count, dtype: int64
Prediction confidence - Mean: 0.9602, Min: 0.5481, Max: 0.9859
Enhanced submission file 'enhanced_submission.csv' created successfully!
Submission shape: (6175, 2)
Sample predictions:
          id Personality
id                      
18524  18524   Extrovert
18525  18525   Introvert
18526  18526   Extrovert
18527  18527   Extrovert
18528  18528   Introvert
18529  18529   Extrovert
18530  18530   Extrovert
18531  18531   Introvert
18532  18532   Extrovert
18533  18533   Introvert
Test probabilities saved to 'test_probabilities.csv'


In [61]:
# Step 10: Advanced Calibration and Threshold Optimization
if len(val_probabilities.columns) > 1:
    try:
        # Get validation probabilities for threshold optimization
        val_probs = val_probabilities.iloc[:, 1].values  # Positive class (adjust index if needed)

        # Advanced threshold optimization
        def optimize_threshold_advanced(y_true, y_prob):
            thresholds = np.arange(0.1, 0.9, 0.005)  # Finer granularity
            best_accuracy = 0
            best_threshold = 0.5
            threshold_scores = []

            # Create label encoder for consistent comparison
            le = LabelEncoder()
            le.fit(['Extrovert', 'Introvert'])  # Ensure consistent mapping
            y_true_encoded = le.transform(y_true)

            for threshold in thresholds:
                y_pred_binary = (y_prob >= threshold).astype(int)
                accuracy = accuracy_score(y_true_encoded, y_pred_binary)
                threshold_scores.append((threshold, accuracy))

                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_threshold = threshold

            return best_threshold, best_accuracy, threshold_scores

        optimal_threshold, optimal_accuracy, threshold_scores = optimize_threshold_advanced(y_val_final, val_probs)

        print(f"Optimal threshold: {optimal_threshold:.4f}")
        print(f"Optimal validation accuracy: {optimal_accuracy:.6f}")

        # Apply optimal threshold to test predictions
        test_probs = test_probabilities.iloc[:, 1].values
        test_predictions_optimized = (test_probs >= optimal_threshold).astype(int)

        # Map back to original labels
        le = LabelEncoder()
        le.fit(['Extrovert', 'Introvert'])
        test_predictions_final = le.inverse_transform(test_predictions_optimized)

        # Create optimized submission
        submission_optimized = pd.DataFrame({
            'id': test.index,
            'Personality': test_predictions_final
        })

        submission_optimized.to_csv('submission_threshold_optimized.csv', index=False)

        print("Threshold-optimized submission created!")
        print("Optimized test prediction distribution:")
        print(pd.Series(test_predictions_final).value_counts())

        # Compare standard vs optimized predictions
        prediction_changes = sum(test_predictions != test_predictions_final)
        print(f"Predictions changed by threshold optimization: {prediction_changes} out of {len(test_predictions)}")

        # Temperature Calibration (additional enhancement)
        class TemperatureScaler:
            def __init__(self):
                self.temperature = 1.0

            def fit(self, logits, y_true):
                from scipy.optimize import minimize_scalar

                def objective(temp):
                    calibrated_probs = self.calibrate_probs(logits, temp)
                    # Use log loss as calibration metric
                    le = LabelEncoder()
                    y_encoded = le.fit_transform(y_true)
                    loss = -np.mean(y_encoded * np.log(calibrated_probs + 1e-15) +
                                   (1 - y_encoded) * np.log(1 - calibrated_probs + 1e-15))
                    return loss

                result = minimize_scalar(objective, bounds=(0.1, 5.0), method='bounded')
                self.temperature = result.x
                return self

            def calibrate_probs(self, logits, temperature):
                return 1 / (1 + np.exp(-logits / temperature))

            def predict_proba(self, logits):
                return self.calibrate_probs(logits, self.temperature)

        # Apply temperature calibration
        temp_scaler = TemperatureScaler()
        # Convert probabilities to logits for calibration
        val_logits = np.log(val_probs / (1 - val_probs + 1e-15))
        temp_scaler.fit(val_logits, y_val_final)

        print(f"Optimal temperature: {temp_scaler.temperature:.4f}")

        # Apply to test set
        test_logits = np.log(test_probs / (1 - test_probs + 1e-15))
        test_probs_calibrated = temp_scaler.predict_proba(test_logits)
        test_predictions_temp_calibrated = (test_probs_calibrated >= optimal_threshold).astype(int)
        test_predictions_temp_final = le.inverse_transform(test_predictions_temp_calibrated)

        # Create temperature calibrated submission
        submission_temp_calibrated = pd.DataFrame({
            'id': test.index,
            'Personality': test_predictions_temp_final
        })

        submission_temp_calibrated.to_csv('submission_temp_calibrated.csv', index=False)
        print("Temperature calibrated submission created!")

    except Exception as e:
        print(f"Advanced calibration failed: {e}")
        print("Using standard predictions")

else:
    print("Skipping calibration - single probability column detected")


Optimal threshold: 0.6600
Optimal validation accuracy: 0.965906
Threshold-optimized submission created!
Optimized test prediction distribution:
Extrovert    4618
Introvert    1557
Name: count, dtype: int64
Predictions changed by threshold optimization: 3 out of 6175
Optimal temperature: 0.9340
Temperature calibrated submission created!


In [63]:
# Step 11: Enhanced Cross-Validation for Performance Verification

!pip install autogluon.tabular[catboost]==1.3.1
from sklearn.model_selection import StratifiedKFold

def enhanced_cv_evaluation(data, n_splits=5):
    """Robust CV evaluation to verify performance and detect overfitting"""
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    cv_scores = []
    cv_detailed_results = []

    print(f"Running {n_splits}-fold cross-validation for performance verification...")
    print("="*60)

    for fold, (train_idx, val_idx) in enumerate(skf.split(data.drop('Personality', axis=1), data['Personality'])):
        print(f"Training fold {fold+1}/{n_splits}...")

        train_fold = data.iloc[train_idx]
        val_fold = data.iloc[val_idx]

        # Use efficient configuration for CV
        fold_predictor = TabularPredictor(
            label='Personality',
            path=f'cv_fold_{fold}',
            verbosity=1
        )

        try:
            fold_predictor.fit(
                TabularDataset(train_fold),
                time_limit=1800,  # 30 minutes per fold
                presets='high_quality',
                hyperparameters={
                    'XGB': {
                        'n_estimators': 200,
                        'max_depth': 8,
                        'learning_rate': 0.05,
                        'scale_pos_weight': class_weight_ratio
                    },
                    'RF': {
                        'n_estimators': 150,
                        'class_weight': 'balanced'
                    },
                    'CAT': {
                        'iterations': 300,
                        'depth': 8,
                        'class_weights': [1, class_weight_ratio]
                    }
                },
                num_bag_folds=3,
                num_stack_levels=2
            )

            # Evaluate fold
            val_result = fold_predictor.evaluate(TabularDataset(val_fold), silent=True)
            val_accuracy = val_result['accuracy']

            # Get predictions for detailed analysis
            val_predictions = fold_predictor.predict(val_fold.drop('Personality', axis=1))
            val_pred_dist = pd.Series(val_predictions).value_counts()
            val_actual_dist = pd.Series(val_fold['Personality']).value_counts()

            cv_scores.append(val_accuracy)
            cv_detailed_results.append({
                'fold': fold + 1,
                'accuracy': val_accuracy,
                'pred_dist': val_pred_dist,
                'actual_dist': val_actual_dist
            })

            print(f"Fold {fold+1} accuracy: {val_accuracy:.6f}")
            print(f"  Predictions: {dict(val_pred_dist)}")
            print(f"  Actual:      {dict(val_actual_dist)}")

        except Exception as e:
            print(f"Fold {fold+1} failed: {e}")
            cv_scores.append(0.0)  # Assign low score to failed folds

    # Calculate CV statistics
    cv_mean = np.mean(cv_scores)
    cv_std = np.std(cv_scores)
    cv_min = np.min(cv_scores)
    cv_max = np.max(cv_scores)

    print("\n" + "="*60)
    print("CROSS-VALIDATION RESULTS SUMMARY")
    print("="*60)
    print(f"Mean Accuracy:        {cv_mean:.6f}")
    print(f"Standard Deviation:   {cv_std:.6f}")
    print(f"Min Accuracy:         {cv_min:.6f}")
    print(f"Max Accuracy:         {cv_max:.6f}")
    print(f"95% Confidence Int:   [{cv_mean - 1.96*cv_std:.6f}, {cv_mean + 1.96*cv_std:.6f}]")

    # Performance assessment
    if cv_mean > 0.980:
        print("🎯 EXCELLENT! CV suggests test performance >98.0%")
    elif cv_mean > 0.975:
        print("✅ VERY GOOD! CV suggests test performance >97.5%")
    elif cv_mean > 0.970:
        print("✅ GOOD! CV suggests test performance >97.0%")
    else:
        print("⚠️  NEEDS IMPROVEMENT - CV suggests <97.0%")

    # Stability check
    if cv_std < 0.005:
        print("📊 Model is VERY STABLE across folds")
    elif cv_std < 0.010:
        print("📊 Model is STABLE across folds")
    else:
        print("⚠️  Model shows variability across folds - consider more regularization")

    return cv_mean, cv_std, cv_detailed_results

# Run enhanced CV evaluation
print("Starting comprehensive cross-validation evaluation...")
cv_mean, cv_std, cv_details = enhanced_cv_evaluation(train_optimized, n_splits=5)

# Final performance prediction
predicted_test_performance = cv_mean
print(f"\n🎯 PREDICTED TEST PERFORMANCE: {predicted_test_performance:.6f}")

if predicted_test_performance > 0.975:
    print("Ready for submission! Expected to exceed target accuracy.")
else:
    print("Consider additional tuning before final submission.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.8 MB/s eta 0:00:00


Starting comprehensive cross-validation evaluation...
Running 5-fold cross-validation for performance verification...
Training fold 1/5...
Fold 1 accuracy: 0.965186
  Predictions: {'Extrovert': np.int64(3032), 'Introvert': np.int64(1133)}
  Actual:      {'Extrovert': np.int64(3021), 'Introvert': np.int64(1144)}
Training fold 2/5...
Fold 2 accuracy: 0.960624
  Predictions: {'Extrovert': np.int64(3025), 'Introvert': np.int64(1140)}
  Actual:      {'Extrovert': np.int64(3021), 'Introvert': np.int64(1144)}
Training fold 3/5...
Fold 3 accuracy: 0.959424
  Predictions: {'Extrovert': np.int64(3016), 'Introvert': np.int64(1149)}
  Actual:      {'Extrovert': np.int64(3021), 'Introvert': np.int64(1144)}
Training fold 4/5...
Fold 4 accuracy: 0.968067
  Predictions: {'Extrovert': np.int64(3008), 'Introvert': np.int64(1157)}
  Actual:      {'Extrovert': np.int64(3021), 'Introvert': np.int64(1144)}
Training fold 5/5...
Fold 5 accuracy: 0.962545
  Predictions: {'Extrovert': np.int64(3043), 'Introvert